In [1]:
import numpy as np
from util.utility_functions import load_dataset

from util.utility_functions import get_random_segment
import librosa
from speechpy.feature import lmfe
from util.detector import calculate_RMS

import matplotlib.pyplot as plt

(11307, 2)


# Load dataset

In [2]:
path_to_file = './audio/'
path_to_file_test = './test/'
path_to_file_unknown = './unknown/'
# extract classes from txt file

raw_classes = np.loadtxt('./meta/meta.txt', dtype='str')
classes = np.delete(raw_classes, [1, 2, 3], 1)

text_labels = ['background', 'bags', 'door', 'keyboard', 'knocking_door', 'ring', 'speech', 'tool']
for index, label in enumerate(text_labels):
    classes[classes == label] = index

dict_classes = dict(classes)
classes_test = np.loadtxt('./meta/meta-test.txt',dtype='str')

dict_classes_test = dict(classes_test)

raw_data_train = np.array(load_dataset(path_to_file, dict_classes))
raw_data_test = np.array(load_dataset(path_to_file_test, dict_classes_test))
raw_data_unknown = np.array(load_dataset(path_to_file_unknown))

print(raw_data_train.shape)

(11307, 2)


# Random 3 sec segmentation

In [ ]:
# random 3 sec segmentation

from util.detector import Detector
from util.utility_functions import segment_audio, segment_audio_unknown

detector = Detector()


segmented_data_train = segment_audio(raw_data_train, detector)

segmented_data_test = segment_audio(raw_data_test, detector)

segmented_data_unknown = segment_audio_unknown(raw_data_unknown)


print(segmented_data_test)

# Extraction features

In [ ]:
from util.utility_functions import extract_feature

mode = 'mbe'
x_train = np.array(extract_feature(segmented_data_train, mode))
x_test = np.array(extract_feature(segmented_data_test, mode))
x_unknown = np.array(extract_feature(segmented_data_unknown, mode))


(137, 298, 40)


In [19]:
label_train = raw_data_train[:,1]
y_train = np.zeros((label_train.shape[0],8))

for i in range(label_train.shape[0]):
    label = int(label_train[i])
    y_train[i, label] = 1
    
label_test= raw_data_test[:,1]
y_test = np.zeros((label_test.shape[0],8))

for i in range(label_test.shape[0]):
    label = int(label_test[i])
    if(label ==8):
        continue
    y_test[i, label] = 1
print(x_unknown.shape)


(137, 298, 40)


In [8]:
print(x_unknown.shape)
print(y_train)

(137, 298, 40)
['2' '2' '7' ... '0' '2' '2']


In [16]:
filename = 'mbe_3sec_random_default'
np.save('data/'+ filename, x_train)
np.save('data/y', y_train)
np.save('data/' + filename+ '_test', x_test)
np.save('data/y_test', y_test)
np.save('data/x_unknown',x_unknown)

#Detector

In [13]:
# extract features for training detector


# test 

# files without events
background = raw_data_test[raw_data_test[:,1] == '0']

background_features_test = []

for frame in background[:,0]:

    sound = get_random_segment(frame)
    feature = lmfe(sound.astype(float), 16000)

    background_features_test.append(feature)


# files with events
events1 = raw_data_test[(raw_data_test[:,1] == '1')]
events2 = raw_data_test[(raw_data_test[:,1] == '2')]
events3 = raw_data_test[(raw_data_test[:,1] == '3')]
events4 = raw_data_test[(raw_data_test[:,1] == '4')]
events5 = raw_data_test[(raw_data_test[:,1] == '5')]
events6 = raw_data_test[(raw_data_test[:,1] == '6')]
events7 = raw_data_test[(raw_data_test[:,1] == '7')]

events = np.concatenate([events1,events2, events3, events4, events5, events6, events7])

event_features_test = []
for frame in events[:,0]:
    # extract feature
    sound = get_random_segment(frame)
    feature = lmfe(sound.astype(float), 16000)

    event_features_test.append(feature)

    pass

print(events.shape)

# train 

background = raw_data_train[raw_data_train[:,1] == '0']

background_features_train= []

for frame in background[:,0]:

    sound = get_random_segment(frame)
    feature = lmfe(sound.astype(float), 16000)

    background_features_train.append(feature)


# files with events
events1 = raw_data_train[(raw_data_train[:,1] == '1')]
events2 = raw_data_train[(raw_data_train[:,1] == '2')]
events3 = raw_data_train[(raw_data_train[:,1] == '3')]
events4 = raw_data_train[(raw_data_train[:,1] == '4')]
events5 = raw_data_train[(raw_data_train[:,1] == '5')]
events6 = raw_data_train[(raw_data_train[:,1] == '6')]
events7 = raw_data_train[(raw_data_train[:,1] == '7')]

events = np.concatenate([events1,events2, events3, events4, events5, events6, events7])

event_features_train = []
for frame in events[:,0]:
    # extract feature
    sound = get_random_segment(frame)
    feature = lmfe(sound.astype(float), 16000)

    event_features_train.append(feature)

    pass


(432, 2)


In [24]:
from sklearn.utils import shuffle


event_features_train = shuffle(event_features_train)
event_features_train = event_features_train[:1000]
x_train_detector = np.concatenate([background_features_train, event_features_train])
x_train_detector = x_train_detector.reshape((-1,x_train_detector.shape[1],x_train_detector.shape[2], 1))
y_train_detector = np.zeros((x_train_detector.shape[0],2))
y_train_detector[:len(background_features_train), 0] = 1
y_train_detector[len(background_features_train):, 1] = 1

event_features_test = shuffle(event_features_test)
event_features_test = event_features_test[:100]
x_test_detector = np.concatenate([background_features_test, event_features_test])
x_test_detector = x_test_detector.reshape((-1,x_test_detector.shape[1], x_test_detector.shape[2], 1))
y_test_detector = np.zeros((x_test_detector.shape[0],2))
y_test_detector[:len(background_features_test), 0] = 1
y_test_detector[len(background_features_test):, 1] = 1


filename = 'detector_data_test'
np.save('data/' + filename, x_train_detector)
np.save('data/y' + filename, y_train_detector)


In [27]:
x_test_detector.shape


(141, 298, 40, 1)